<img src="http://www.finanze.gov.it/export/system/modules/it.gov.finanze.template/resources/img/logo-mef.gif"  style="float: left; width: 20%; margin-left: 18%; margin-top: 0.5em;">

<img src="http://www.opencoesione.gov.it/static/img/header/testata.png" style="float: right; width: 35%; margin-right: 18%; margin-top: 1.2em;">

<p style="clear: both;">

## Studio a livello comunale sui finanziamenti a enti privati da dati OpenCoesione

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import gc #garbage collector
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,12)
%config Completer.use_jedi = False #fix TAB slowness with big frames

### Load and prepare datasets

In [ ]:
# --- Spese e redditi da OpenCivitas
speseERedditiComune = pd.read_csv('../opencivitas/data/opencivitas_spesa_storica_dati_irpef.csv') #dati per anno

provToRegioni = pd.read_csv('../provToReg.csv', dtype='str', index_col='Sigla automobilistica')
speseERedditiComune = speseERedditiComune[~speseERedditiComune['Codice Istat Comune'].isna()]
speseERedditiComune = pd.merge(
    speseERedditiComune, provToRegioni, how='left', left_on='Sigla Provincia_x',right_index=True)

padZeros = np.vectorize(lambda x: str(x).rjust(6, '0')) # some utils

speseERedditiComune['Codice Istat Lungo'] = \
(speseERedditiComune['Codice regione'] + padZeros(speseERedditiComune['Codice Istat Comune'].astype(int))).astype(int)

# --- Soggetti e luoghi da OpenCoesione
soggettiLuoghi = pd.read_csv('../pac_opencoesione/SoggettiELuoghi.csv', low_memory=False)
soggettiLuoghi.drop(columns=['SOGG_COD_RUOLO', 'SOGG_PROGR_RUOLO'], inplace=True)
soggettiLuoghi = soggettiLuoghi[~soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.isna()]
assert not any(soggettiLuoghi['COD_COMUNE_SEDE_SOGG'].isna()), 'Unexpected missing codes'
soggettiLuoghi['COD_COMUNE_SEDE_SOGG'] = soggettiLuoghi['COD_COMUNE_SEDE_SOGG'].astype(int)
soggettiLuoghi['COD_COMUNE_LUOGO'] = soggettiLuoghi['COD_COMUNE_LUOGO'].fillna(value=-1).astype(int)

### Join OpenCoesione and OpenCivitas

In [ ]:
# --- Anagrafica Comuni
path_comuni = '../pac_agenziaentrate/regioni_citta/comuni/archivio_comuni/ElencoComuniAttuali_20170918.csv.read'

comuniAnagrafica = pd.read_csv(path_comuni, sep=';')
comuniAnagrafica.set_index('Codice Istat', inplace=True)
comuniAnagrafica = comuniAnagrafica[~comuniAnagrafica.index.isnull()]
assert not any(comuniAnagrafica.index.duplicated()), 'Duplicate Istat codes'
convTable = pd.read_csv('../provToReg.csv', index_col='Sigla automobilistica', dtype='str')
comuniAnagrafica = comuniAnagrafica.join(convTable, on='Sigla Provincia')
formattedValues = comuniAnagrafica.index.values.astype('int')
padZeros = np.vectorize(lambda x: str(x).rjust(6, '0'))
comuniAnagrafica['Codice Istat Lungo'] = (comuniAnagrafica['Codice regione'] + padZeros(formattedValues)).astype(int)

In [ ]:
#speseERedditiComune.COMUNE_CAT_COD.isin(comuniAnagrafica['Codice Nazionale'].values).mean()
#speseERedditiComune.join(comuniAnagrafica['Codice Istat Lungo'])

soggettiLuoghi.COD_COMUNE_SEDE_SOGG[~soggettiLuoghi.COD_COMUNE_SEDE_SOGG.isin(speseERedditiComune['Codice Istat Lungo'])]


In [ ]:
bEntePrivato = soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.str.startswith('1') & (soggettiLuoghi.COD_COMUNE_SEDE_SOGG != -1)

soggLuoghiPrivato = soggettiLuoghi[bEntePrivato]

totPagamentiPrivato = soggLuoghiPrivato.TOT_PAGAMENTI.sum()
print(totPagamentiPrivato)

comuneCodesPrivato = soggLuoghiPrivato.COD_COMUNE_SEDE_SOGG.drop_duplicates()
mismatching = comuneCodesPrivato[(comuneCodesPrivato.isin(comuniAnagrafica['Codice Istat Lungo'])) & ~(
        comuneCodesPrivato.isin(speseERedditiComune['Codice Istat Lungo']))]


#print(soggLuoghiPrivato.COD_COMUNE_SEDE_SOGG.drop_duplicates().isin(speseERedditiComune['Codice Istat Lungo']).mean())
#print(soggLuoghiPrivato.COD_COMUNE_LUOGO.drop_duplicates().isin(speseERedditiComune['Codice Istat Lungo']).mean())
#print(soggLuoghiPrivato.COD_COMUNE_SEDE_SOGG.drop_duplicates().isin(comuniAnagrafica['Codice Istat Lungo']).mean())
#print(comuniAnagrafica['Codice Istat Lungo'].drop_duplicates().isin(speseERedditiComune['Codice Istat Lungo']).mean())


# luoghi (comuni) di sede soggetto che hanno ricevuto piu' fondi
fondiRicevuti = soggLuoghiPrivato.groupby('COD_COMUNE_SEDE_SOGG')['TOT_PAGAMENTI'].sum().sort_values(ascending=False).to_frame()
newData = fondiRicevuti.join(comuniAnagrafica.set_index('Codice Istat Lungo'))

listaComuniLeft = speseERedditiComune['Codice Istat Lungo'].drop_duplicates()

print(listaComuniLeft.isin(fondiRicevuti.index).mean(), comuniAnagrafica['Codice Istat Lungo'].isin(fondiRicevuti.index).mean())
print( fondiRicevuti.index.drop_duplicates().isin(listaComuniLeft).mean(), comuniAnagrafica['Codice Istat Lungo'].isin(listaComuniLeft).mean())

print( listaComuniLeft.isin(comuniAnagrafica['Codice Istat Lungo']).mean())
print( fondiRicevuti.index.isin(comuniAnagrafica['Codice Istat Lungo']).mean())


print(fondiRicevuti[~fondiRicevuti.index.isin(comuniAnagrafica['Codice Istat Lungo'])].sum())

fondiRicevuti[fondiRicevuti.index.isin(listaComuniLeft)]

In [ ]:
unioneComuniSoggetti = pd.merge(
    speseERedditiComune, 
    fondiRicevuti, 
    how='left',
    left_on='Codice Istat Lungo',
    right_index = True)


#unioneComuniSoggetti.TOT_PAGAMENTI.fillna(value=0, inplace=True)
#print(newData.COD_COMUNE_SEDE_SOGG.drop_duplicates().isin(speseERedditiComune['Codice Istat Lungo']).mean())


unioneComuniSoggetti.ANNO.unique()
unioneComuniSoggetti[newData.ANNO==2013].TOT_PAGAMENTI.sum()


In [ ]:
#newData.sort_values('TOT_PAGAMENTI', ascending=False)

ax = plt.scatter(np.log10(unioneComuniSoggetti.SPESA_CCC_tot), np.log10(unioneComuniSoggetti.TOT_PAGAMENTI+1))
plt.show()


In [ ]:
bDodgyDataPublic = soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.str.startswith('2') & (soggettiLuoghi.COD_COMUNE_SEDE_SOGG != -1)
#soggettiLuoghi[bDodgyDataPublic].sort_values('TOT_PAGAMENTI', ascending=False)
#soggettiLuoghi['TOT_PAGAMENTI'][bDodgyData].sort_values(ascending=False)
soggettiLuoghi[bDodgyDataPublic].DESCR_FORMA_GIURIDICA_SOGG.unique()

In [ ]:
bDodgyData = soggettiLuoghi.COD_FORMA_GIURIDICA_SOGG.str.startswith('1') & (soggettiLuoghi.COD_COMUNE_SEDE_SOGG == -1)
soggettiLuoghi[bDodgyData].sort_values('TOT_PAGAMENTI', ascending=False)
soggettiLuoghi[bDodgyData].DESCR_FORMA_GIURIDICA_SOGG.unique()